In [1]:
import cartopy
import numpy as np

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from rasterio.warp import transform

In [2]:
xi, yi = [204, 1097], [219, 1269]

da = xr.open_rasterio('https://noaa-nbm-pds.s3.amazonaws.com/blendv4.0/conus/' +
                      '2020/10/08/1200/probqpf24gt001/blendv4.0_conus_probqpf24gt001' + 
                      '_2020-10-08T12%3A00_2020-10-09T12%3A00.tif',
                     ).isel(band=0, x=slice(xi[0], xi[1]), y=slice(yi[0], yi[1])).drop('band')

# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da['y']), len(da['x'])
x, y = np.meshgrid(da['x'], da['y'])

# Rasterio works with 1D arrays
lon, lat = transform(da.crs, {'init': 'EPSG:4326'},
                     x.flatten(), y.flatten())
lon = np.asarray(lon).reshape((ny, nx))
lat = np.asarray(lat).reshape((ny, nx))
da.coords['lon'] = (('y', 'x'), lon)
da.coords['lat'] = (('y', 'x'), lat)

da.load()

NameError: name 'xr' is not defined

In [ ]:
fig = plt.figure(figsize=(20, 12))
ax = plt.subplot(projection=ccrs.PlateCarree())
da.plot(ax=ax, x='lon', y='lat', transform=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.STATES)